In [2]:
# Step 1: User inputs 
# img_path = "/Users/mandylee/Documents/AI/baby.jpg"
img_path = '/Users/mandylee/Documents/AI/GenAIProject/ArtiFace/Fake/StarGAN/StarGAN.2324.jpg'

In [3]:
import tensorflow as tf
import numpy as np
import cv2
import joblib  # For loading the meta-classifier
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

# Load trained models
# Load MobileNetV2
mobilenet_model = tf.keras.models.load_model("/Users/mandylee/Documents/AI/GenAIProject/MarchRuns/Metaclassifier/mobilenet_final_model-finetuned2.keras")

# Load ViT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/Users/mandylee/Documents/AI/GenAIProject/MarchRuns/Metaclassifier/model2_complete.pth"
vit_model = torch.load(model_path, map_location=device, weights_only=False)
vit_model.to(device)

# Load Metaclassifier
meta_classifier = joblib.load("/Users/mandylee/Documents/AI/GenAIProject/MarchRuns/Metaclassifier/meta_classifierprob.pkl")

# Preprocessing for MobileNetV2 (Single Image)
def preprocess_for_mobilenet(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (224, 224))  # Resize to MobileNetV2 input size
    image = image.astype("float32") / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension (1, 224, 224, 3)
    return image

# Preprocessing for ViT (Single Image)
def preprocess_for_vit(img_path):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension (1, 3, 224, 224)
    return image.to(device)

# Function to predict 
def predict_real_or_fake(img_path):

    # Get MobileNetV2 probabilities
    mobilenet_input = preprocess_for_mobilenet(img_path)
    mobilenet_probs = mobilenet_model.predict(mobilenet_input)  # Probability of Fake
    mobilenet_probs = np.hstack([1 - mobilenet_probs, mobilenet_probs])  # Convert to [Real prob, Fake prob]
    print(mobilenet_probs)

    # Get ViT Probabilities
    vit_input = preprocess_for_vit(img_path)
    with torch.no_grad():
        outputs = vit_model(vit_input)
        vit_probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()  # Convert to probabilities [Real prob, Fake prob]
        print(vit_probs)

    # Combine all probabilities [Real_ViT, Fake_ViT, Real_MobileNet, Fake_MobileNet]
    new_data = pd.DataFrame({
        "ViT_Prob_Real": [vit_probs[0, 0]],   # ViT probability of real
        "ViT_Prob_Fake": [vit_probs[0, 1]],   # ViT probability of fake
        "MobileNet_Prob_Real": [mobilenet_probs[0, 0]],  # MobileNet probability of real
        "MobileNet_Prob_Fake": [mobilenet_probs[0, 1]]   # MobileNet probability of fake
    })

    # Get final prediction
    final_prediction = meta_classifier.predict(new_data)[0]  # 0 = Real, 1 = Fake
    confidence = meta_classifier.predict_proba(new_data)[0][final_prediction]  # Confidence score

    result = "Fake" if final_prediction == 1 else "Real"
    print(f"Final Prediction: {result} (Confidence: {confidence:.2f})")

predict_real_or_fake(img_path)

2025-03-31 21:16:08.078427: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-03-31 21:16:08.078458: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-31 21:16:08.078464: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1743426968.078498  424065 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1743426968.078536  424065 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-03-31 21:16:09.776037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
E0000 00:00:1743426969.831675  424065 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.13314795 0.86685205]]
[[5.0407014e-04 9.9949586e-01]]
Final Prediction: Fake (Confidence: 0.99)
